In [167]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score as r2

In [168]:
train = pd.read_csv('train.csv')
train.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [169]:
train_copy = train.copy()

In [170]:
def Correct_Square(train_copy):
    ID_List = []
    ID_List.extend(train_copy.loc[train_copy['LifeSquare'] > train_copy['Square']].index)
    for i in ID_List:
        if train_copy.loc[i]['LifeSquare'] > -1:
            temp = train_copy.loc[i]['LifeSquare']
            train_copy.loc[i, 'LifeSquare'] = train_copy.loc[i]['Square']
            train_copy.loc[i, 'Square'] = temp
    train_copy.loc[train_copy.LifeSquare.isnull(), 'LifeSquare'] = train_copy.loc[train_copy.LifeSquare.isnull(), 'LifeSquare'].fillna(train_copy.Square - (train_copy.Square * 10)/100)
    train_copy.loc[train_copy.LifeSquare > (train_copy.loc[train_copy.Rooms, 'LifeSquare'].mean() + 100), 'LifeSquare'] = train_copy.loc[train_copy.Rooms, 'LifeSquare'].mean()
    
    
    
    

In [171]:
def Correct_floors(train_copy):
    ID_List = []
    ID_List.extend(train_copy.loc[train_copy['Floor'] > train_copy['HouseFloor']].index)
    for i in ID_List:
        if train_copy.loc[i]['Floor'] > -1:
            temp = train_copy.loc[i]['Floor']
            train_copy.loc[i, 'Floor'] = train_copy.loc[i]['HouseFloor']
            train_copy.loc[i, 'HouseFloor'] = temp

In [172]:
def Correct_other(train_copy):
    train_copy.loc[train_copy['HouseYear'] > 2019, 'HouseYear'] = train_copy['HouseYear'].median()
    train_copy.loc[train_copy.KitchenSquare > train_copy.Square, 'KitchenSquare'] = train_copy.loc[train_copy.KitchenSquare > train_copy.Square]['Square'] - train_copy.loc[train_copy.KitchenSquare > train_copy.Square]['LifeSquare']
    train_copy.loc[train_copy.HouseFloor > 50, 'HouseFloor'] = train_copy.loc[train_copy.HouseFloor > 50]['Floor'].median() + 10
    train_copy.loc[(train_copy.Rooms > 5) & (train_copy.loc[train_copy.Rooms > 5]['Square'] < train_copy.loc[train_copy.Rooms == 5]['Square'].mean()), 'Rooms'] = train_copy.loc[train_copy.Rooms]['Rooms'].median() 
    train_copy.loc[train_copy.Square > (train_copy.loc[train_copy.Rooms, 'Square'].mean() + 100), 'Square'] = train_copy.loc[train_copy.Rooms, 'Square'].mean()
    

In [173]:
features = ['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2', 'Social_3', 'Helthcare_2',
       'Shops_1']
target = ['Price']

In [174]:
 def prepare_features(train_copy):
    Correct_Square(train_copy)
    Correct_floors(train_copy)
    Correct_other(train_copy)



In [175]:
prepare_features(train_copy)

In [176]:
X_train, X_valid, y_train, y_valid = train_test_split(train_copy[features], train_copy[target], \
                                                      test_size=0.3, random_state=42)

In [177]:
model = RFR(n_estimators=5, max_depth=12, random_state=42)

In [178]:
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=12,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [179]:
model.feature_importances_

array([0.01687277, 0.04624536, 0.09021432, 0.40565644, 0.02756244,
       0.02194005, 0.01619743, 0.0280142 , 0.0280757 , 0.04264222,
       0.09785502, 0.09831768, 0.05871757, 0.00393866, 0.01775015])

In [180]:
 y_pred = model.predict(X_valid)

In [181]:
r2(y_valid, y_pred)

0.6656139999109631

In [213]:
def Testing_models():
    
    r2_value = 0
    estimator = 0
    depth = 0
    
    for n_estimators in range(1, 16):
        for max_depth in range(1, 25, 2):
            model = RFR(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_valid)
            if r2_value < r2(y_valid, y_pred):
                r2_value = r2(y_valid, y_pred)
                estimator = n_estimators
                depth = max_depth
           
    
    return estimator, depth, r2_value

In [214]:
def Testing_models_extended(X_train, X_valid, y_train, y_valid):
    
    r2_value = 0
    estimator = 0
    depth = 0
    
    for n_estimators in range(1, 16):
        for max_depth in range(1, 25, 2):
            model = RFR(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_valid)
            if r2_value < r2(y_valid, y_pred):
                r2_value = r2(y_valid, y_pred)
                estimator = n_estimators
                depth = max_depth
           
    
    return estimator, depth, r2_value

In [215]:
n_estimator, max_depth, r2_value = Testing_models()

In [216]:
n_estimator, max_depth, r2_value

(15, 13, 0.6949417968553182)

In [217]:
best_r2_yet = r2_value

In [218]:
final_model = RFR(n_estimators=n_estimator, max_depth=max_depth, random_state=42)

In [219]:
from collections import deque
test_features = deque()


In [220]:

def Testing_features(features, best_r2_yet):

#     r2_best = -100
#     best_feat_list = [None for i in range(len(features))]
#     test_features = features.copy()
#     final_list = []
    
    
#     for i in range(len(test_features)):
        
#         best_feat = None
        
#         for j in range(len(test_features)):
            
#             if not(test_features[j] in best_feat_list):
                
#                 best_feat_list[i] = test_features[j]

#             else:
#                 continue
            
#             final_model.fit(X_train[best_feat_list[:i+1]], y_train)
#             y_pred = final_model.predict(X_valid[best_feat_list[:i+1]])
            
#             if r2_best < r2(y_valid, y_pred):
#                 r2_best = r2(y_valid, y_pred)
#                 best_feat = test_features[j]

        
#         if best_feat != None:
# #             print(best_feat, i)
#             print(r2_best)
#             best_feat_list[i] = best_feat
#             final_list.append(best_feat)
       
        
#     return final_list, r2_best


# отбор по убыванию

    
    flag = True
    r2_best = 0
    test_features = deque(features.copy())
    del_list = []
    must_del_feat = None
    best_r2_yet = best_r2_yet
    
    while flag:
        
        for i in range(len(test_features)):
            
            del_list.append(test_features.pop())
            
            final_model.fit(X_train[test_features], y_train)
            y_pred = final_model.predict(X_valid[test_features])
            
            if r2_best < r2(y_valid, y_pred):
                
                r2_best = r2(y_valid, y_pred)
                
                must_del_feat = del_list[0]
                
            test_features.appendleft(del_list.pop())
        
        if best_r2_yet < r2_best:
            
            best_r2_yet = r2_best
            test_features.remove(must_del_feat)
            
        else:
            flag = False
                
    
    
    return test_features, best_r2_yet


# Тестирование

    
#     test_features = deque(features.copy())
#     del_list = []
#     best_r2_yet = best_r2_yet
    
#     for i in range(len(test_features)):
        
#         del_list.append(test_features.pop())
#         final_model.fit(X_train[test_features], y_train)
#         y_pred = final_model.predict(X_valid[test_features])
        
        
#         if best_r2_yet < r2(y_valid, y_pred):
            
#             print('Yes')
            
#         else:
#             print(best_r2_yet)
#             print(r2(y_valid, y_pred))
#             print(test_features, len(test_features), del_list[0])
        
#         test_features.appendleft(del_list.pop())
    
    
    
    
    
    

In [221]:

Testing_features(features, best_r2_yet)


(deque(['Id',
        'DistrictId',
        'Rooms',
        'Square',
        'KitchenSquare',
        'Floor',
        'HouseFloor',
        'HouseYear',
        'Ecology_1',
        'Social_1',
        'Social_3',
        'Helthcare_2',
        'Shops_1']),
 0.7053912279871515)

In [222]:
all_feattures = train_copy[train_copy.columns[:-1]].copy()

In [223]:
all_feattures.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,14038,35,2.0,47.981561,29.442751,6.0,7.0,9.0,1969.0,0.089040,B,B,33,7976,5,NaN,0,11,B
1,15053,41,3.0,65.683640,40.049543,8.0,7.0,9.0,1978.0,0.000070,B,B,46,10309,1,240.0,1,16,B
2,4765,53,2.0,44.947953,29.197612,0.0,8.0,12.0,1968.0,0.049637,B,B,34,7759,0,229.0,1,3,B
3,5809,58,2.0,53.352981,52.731512,9.0,8.0,17.0,1977.0,0.437885,B,B,23,5735,3,1084.0,0,5,B
4,10783,99,1.0,39.649192,23.776169,7.0,11.0,12.0,1976.0,0.012339,B,B,35,5776,1,2078.0,2,4,B


In [224]:
all_feattures = pd.get_dummies(all_feattures)

In [225]:
all_feattures.drop('Healthcare_1', axis=1, inplace=True)

In [226]:
all_feattures.columns

Index(['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2',
       'Social_3', 'Helthcare_2', 'Shops_1', 'Ecology_2_A', 'Ecology_2_B',
       'Ecology_3_A', 'Ecology_3_B', 'Shops_2_A', 'Shops_2_B'],
      dtype='object')

In [227]:
all_feattures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       10000 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null float64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null float64
Ecology_1        10000 non-null float64
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Ecology_2_A      10000 non-null uint8
Ecology_2_B      10000 non-null uint8
Ecology_3_A      10000 non-null uint8
Ecology_3_B      10000 non-null uint8
Shops_2_A        10000 non-null uint8
Shops_2_B        10000 non-null uint8
dtypes: float64(8), int64(7), uint8(6)
memory usage: 1.2 MB


In [228]:
all_feattures.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_2,Social_3,Helthcare_2,Shops_1,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
0,14038,35,2.0,47.981561,29.442751,6.0,7.0,9.0,1969.0,0.089040,...,7976,5,0,11,0,1,0,1,0,1
1,15053,41,3.0,65.683640,40.049543,8.0,7.0,9.0,1978.0,0.000070,...,10309,1,1,16,0,1,0,1,0,1
2,4765,53,2.0,44.947953,29.197612,0.0,8.0,12.0,1968.0,0.049637,...,7759,0,1,3,0,1,0,1,0,1
3,5809,58,2.0,53.352981,52.731512,9.0,8.0,17.0,1977.0,0.437885,...,5735,3,0,5,0,1,0,1,0,1
4,10783,99,1.0,39.649192,23.776169,7.0,11.0,12.0,1976.0,0.012339,...,5776,1,2,4,0,1,0,1,0,1


In [229]:
# all_X_train, all_X_valid, all_y_train, all_y_valid = train_test_split(all_feattures, train_copy[target], \
#                                                       test_size=0.3, random_state=42)

# all_feat_model = RFR(n_estimators=n_estimator, max_depth=max_depth, random_state=42)

In [230]:
all_feat_model.fit(all_X_train, all_y_train)
all_feat_model.feature_importances_


array([2.60920377e-02, 4.99016215e-02, 7.36661163e-02, 3.85562180e-01,
       3.47560751e-02, 2.24099842e-02, 2.08768732e-02, 2.99695687e-02,
       3.30288031e-02, 4.28184619e-02, 9.95747288e-02, 9.19208694e-02,
       6.00650957e-02, 6.80730814e-03, 2.04423163e-02, 2.97470240e-04,
       1.20102205e-04, 2.12958846e-04, 6.01388547e-04, 3.89051780e-04,
       4.86988399e-04])

In [231]:

all_y_predict = all_feat_model.predict(all_X_valid)
all_r2_best_yet = r2(all_y_valid, all_y_predict)
all_r2_best_yet

0.6888321087701919

In [232]:
def Test_all_feats(features, best_r2_yet):
    flag = True
    r2_best = best_r2_yet
    test_features = deque(features.copy())
    del_list = []
    must_del_feat = None
    final_list = []
    
    
    while flag:
        
        for i in range(len(test_features)):
            
#             del_list.append(test_features.pop())
#             all_feat_model.fit(all_X_train[test_features], all_y_train)
#             y_pred = all_feat_model.predict(all_X_valid[test_features])
            
            del_list.append(test_features.pop())
            
            all_X_train, all_X_valid, all_y_train, all_y_valid = train_test_split(all_feattures[test_features], train_copy[target], \
                                                      test_size=0.3, random_state=42)

            all_feat_model = RFR(n_estimators=n_estimator, max_depth=max_depth, random_state=42)
    
            
            
            all_feat_model.fit(all_X_train, all_y_train)
            
            y_pred = all_feat_model.predict(all_X_valid)
    
    
    
            
            if r2_best < r2(all_y_valid, y_pred):
                
                r2_best = r2(all_y_valid, y_pred)
                
                must_del_feat = del_list[0]
                
                print(r2(all_y_valid, y_pred))
                print(len(test_features))
                print(test_features)
                final_list = test_features.copy()
                
            test_features.appendleft(del_list.pop())
        
        if best_r2_yet < r2_best:
            
            best_r2_yet = r2_best
            test_features.remove(must_del_feat)
            
#             if best_r2_yet == 0.716522622569344:
#                 print(test_features)
                
            
        else:
            flag = False
                
#     print(best_r2_yet)
#     print(len(test_features))
#     print(test_features)
    
    return final_list, best_r2_yet, len(test_features)

In [233]:
optimal_features, R2_score_, feat_list_len = Test_all_feats(all_feattures, all_r2_best_yet)
optimal_features

0.6914946264700401
20
deque(['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2', 'Social_3', 'Helthcare_2', 'Shops_1', 'Ecology_2_A', 'Ecology_2_B', 'Ecology_3_A', 'Ecology_3_B', 'Shops_2_A'])
0.6949417968553182
20
deque(['Shops_2_A', 'Shops_2_B', 'Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2', 'Social_3', 'Helthcare_2', 'Shops_1', 'Ecology_2_A', 'Ecology_2_B', 'Ecology_3_A'])
0.6954861049100097
19
deque(['Social_3', 'Helthcare_2', 'Shops_1', 'Ecology_2_A', 'Ecology_2_B', 'Ecology_3_A', 'Shops_2_A', 'Shops_2_B', 'Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1'])
0.695707935234126
19
deque(['Social_2', 'Social_3', 'Helthcare_2', 'Shops_1', 'Ecology_2_A', 'Ecology_2_B', 'Ecology_3_A', 'Shops_2_A', 'Shops_2_B', 'Id', 'District

deque(['Square',
       'KitchenSquare',
       'Floor',
       'HouseYear',
       'Ecology_1',
       'Social_3',
       'Helthcare_2',
       'Shops_1',
       'Ecology_2_A',
       'Ecology_2_B',
       'Ecology_3_A',
       'Shops_2_A',
       'DistrictId'])

In [234]:
R2_score_, feat_list_len
# (0.716522622569344, 17

(0.721864777729562, 13)

In [235]:
train_last_copy = train.copy()
train_last_copy = pd.get_dummies(train_last_copy)
optimal_features = list(optimal_features)
prepare_features(train_last_copy)
train_last_copy.drop('Healthcare_1', axis=1, inplace=True)
X_train, X_valid, y_train, y_valid = train_test_split(train_last_copy[optimal_features], train_last_copy[target], \
                                                      test_size=0.3, random_state=42)
train_last_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 22 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       10000 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null float64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null float64
Ecology_1        10000 non-null float64
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Price            10000 non-null float64
Ecology_2_A      10000 non-null uint8
Ecology_2_B      10000 non-null uint8
Ecology_3_A      10000 non-null uint8
Ecology_3_B      10000 non-null uint8
Shops_2_A        10000 non-null uint8
Shops_2_B        10000 non-null uint8
dtypes: float64(9), int64(7), uint8(6)

In [236]:
n_estimator, max_depth, r2_value = Testing_models_extended(X_train, X_valid, y_train, y_valid)

In [237]:
n_estimator, max_depth, r2_value

(15, 13, 0.721864777729562)